In [3]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from openai.embeddings_utils import cosine_similarity

import tiktoken

# Load environment variables (contains OPENAI_API_BASE and OPENAI_API_KEY)
load_dotenv(override=True)

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE') # looks like https://********.openai.azure.com/
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize embedding model
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002', deployment = 'text-embedding-ada-002',  chunk_size=1)

text_to_encode = 'Cosa sono e a cosa servono gli embeddings?'
textEmbedded = embeddings.embed_query(text_to_encode)
print(textEmbedded)
print(f'lunghezza vettore: {len(textEmbedded)}')

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

embedding_model = 'cl100k_base'
token_count = num_tokens_from_string(text_to_encode, embedding_model)
print(f'frase + embeddings - token utilizzati: {token_count}')
token_count = num_tokens_from_string('Cosa sono e a cosa servono gli hamburgher?', embedding_model)
print(f'frase + hamburgher - token utilizzati: {token_count}')

[-0.014261458646306463, -0.004674444324689038, 0.022025016520924055, -0.02348639198937124, -0.03457718629218908, -0.004782090310480618, 0.00724163726618141, -0.0098838561554381, -0.018136713203480787, -0.007567837180792748, 0.007359069123682786, 0.022729606734609758, -0.0132241432065524, 0.009923000573599836, 0.005020215982719968, -0.0012795190242004379, 0.016544857582924555, 0.027165926318381998, 0.013361147273134642, -0.006400041588929638, -0.0007172319859996458, 0.007652649000266862, -0.003956804419411839, -0.013341574598392499, 0.005036526234564236, -0.003712154599868654, 0.019193600386654443, -0.009583752364380827, 0.012134635519690205, -0.04822539045875716, 0.026878870356271116, -0.014509370190255609, -0.0226252241030386, -0.007306877342235933, 0.010634115633081285, 0.0016089808960483745, -0.0023877831980037103, -0.013230667121025596, 0.011110367908882536, 0.018423769165591665, 0.008826968972264445, 0.011162560155990665, 0.014509370190255609, -0.03126299676161268, -0.023029710525

# Embeddings vector explanation

In [2]:
def printEmbeddings(iText: str):
    # Embed a single word
    e = embeddings.embed_query(iText)
    #print(e)
    print(f'{iText} -> lunghezza stringa: {len(iText)} - lunghezza vettore: {len(e)}')

printEmbeddings("man")
printEmbeddings("king")
printEmbeddings("woman")

man -> lunghezza stringa: 3 - lunghezza vettore: 1536
king -> lunghezza stringa: 4 - lunghezza vettore: 1536
woman -> lunghezza stringa: 5 - lunghezza vettore: 1536


# Embeddings with words

In [15]:
def get_similarity_from_text(text1, text2, varNameText1: str = "", varNameText2: str = "", printText: bool = True):
    e1 = embeddings.embed_query(text1)
    e2 = embeddings.embed_query(text2)
    s = cosine_similarity(e1, e2)

    print(f"Somiglianza tra '{text1 if printText else varNameText1}' e '{text2 if printText else varNameText2}': {s}")


get_similarity_from_text("ragazzo", "bambino")
get_similarity_from_text("ragazzo", "fratello")
get_similarity_from_text("ragazzo", "uomo")
get_similarity_from_text("ragazzo", "ragazza")

Somiglianza tra 'ragazzo' e 'bambino': 0.8620923331020662
Somiglianza tra 'ragazzo' e 'fratello': 0.8694296884442343
Somiglianza tra 'ragazzo' e 'uomo': 0.8787322926416516
Somiglianza tra 'ragazzo' e 'ragazza': 0.9371455305554139


# Confronto diagnosi

In [4]:
case1 = """Paziente: Maschio, 45 anni
Descrizione dei sintomi: Febbre persistente, affaticamento, tosse secca, perdita di peso involontaria e sudorazione notturna.
Diagnosi: Dopo un'attenta valutazione dei sintomi riportati dal paziente, esami fisici e risultati di test di laboratorio, la diagnosi principale è sospetta tubercolosi polmonare.
Motivazione della diagnosi: La febbre persistente, l'affaticamento, la tosse secca e la sudorazione notturna sono sintomi tipici della tubercolosi polmonare. La perdita di peso involontaria è un segno comune nelle fasi avanzate della malattia. La tubercolosi polmonare è una condizione infettiva causata dal batterio Mycobacterium tuberculosis.
Ulteriori indagini: Saranno necessari ulteriori esami per confermare la diagnosi. Si raccomanda una radiografia del torace, una coltura del muco bronchiale e un test cutaneo alla tubercolina per valutare la presenza dell'infezione.
Piano di trattamento: Il paziente dovrà iniziare immediatamente una terapia antibiotica specifica per la tubercolosi polmonare. Sarà necessario un trattamento di lunga durata, generalmente composto da una combinazione di farmaci antibatterici. Il paziente sarà monitorato regolarmente per valutare la risposta al trattamento e gestire eventuali effetti collaterali dei farmaci.
Follow-up: Il paziente dovrà essere sottoposto a controlli periodici per monitorare l'efficacia del trattamento e l'eventuale comparsa di complicanze. Sarà importante mantenere uno stretto contatto con il medico curante e seguire attentamente le istruzioni riguardanti l'assunzione dei farmaci e il controllo degli effetti collaterali."""

case2 = """Paziente: Femmina, 35 anni
Descrizione dei sintomi: Dolore addominale acuto, localizzato nella parte inferiore destra dell'addome, nausea e lieve febbre.
Diagnosi: Dopo una valutazione dei sintomi e un esame fisico, la diagnosi principale è una possibile appendicite acuta.
Motivazione della diagnosi: Il dolore addominale acuto, localizzato nella parte inferiore destra, è un sintomo tipico dell'appendicite acuta. La nausea e la lieve febbre possono essere associati a un'infiammazione dell'appendice.
Ulteriori indagini: Saranno necessari ulteriori esami per confermare la diagnosi. Si consiglia una serie di esami del sangue per valutare i livelli di globuli bianchi e una ecografia addominale per valutare la presenza di infiammazione dell'appendice.
Piano di trattamento: Il paziente dovrà essere sottoposto a un intervento chirurgico d'urgenza per rimuovere l'appendice infiammata (appendicectomia). Questa procedura viene generalmente eseguita laparoscopicamente, consentendo una rapida ripresa postoperatoria.
Follow-up: Dopo l'intervento chirurgico, il paziente dovrà seguire le istruzioni del medico per la gestione del dolore, la cura delle ferite e la ripresa delle normali attività. Saranno programmati appuntamenti di follow-up per valutare la guarigione e risolvere eventuali complicanze postoperatorie."""

case3 = """Paziente: Maschio, 45 anni
Descrizione dei sintomi: Dolore persistente all'addome superiore, affaticamento, perdita di appetito, lieve nausea e senso di gonfiore addominale.
Diagnosi: Dopo una valutazione dei sintomi riportati dalla paziente, esami fisici e risultati di test di laboratorio, la diagnosi principale è sospetta gastrite cronica.
Motivazione della diagnosi: Il dolore persistente all'addome superiore, la perdita di appetito, la lieve nausea e il senso di gonfiore addominale sono sintomi comuni della gastrite cronica. Questa condizione è caratterizzata da un'infiammazione della mucosa dello stomaco.
Ulteriori indagini: Saranno necessari ulteriori esami per confermare la diagnosi. Si raccomanda una gastroscopia per visualizzare direttamente la mucosa gastrica e prelevare campioni per l'analisi dei tessuti.
Piano di trattamento: Il paziente dovrà seguire una dieta bland-a e evitare cibi piccanti, alcolici e caffeina. Sarà prescritta una terapia farmacologica per ridurre l'infiammazione dello stomaco e alleviare i sintomi. In alcuni casi, potrebbe essere necessario un trattamento a lungo termine.

Follow-up: Il paziente dovrà effettuare controlli periodici per monitorare la risposta al trattamento e apportare eventuali modifiche alla terapia. Sarà importante seguire attentamente le indicazioni del medico curante e adottare uno stile di vita sano, evitando fattori che possono aggravare la gastrite cronica."""


get_similarity_from_text(case1, case2, "case1", "case2", False)
get_similarity_from_text(case2, case3, "case2", "case3", False)

Somiglianza tra 'case1' e 'case2': 0.8302775799441025
Somiglianza tra 'case2' e 'case3': 0.905902908732044


# Altri esempi

In [5]:
cv1 = """{
  "nome": "Nome Cognome",
  "sommario": "Sviluppatore .NET con più di 5 anni di esperienza nella progettazione e implementazione di soluzioni software innovative. Leader tecnico con esperienza nella gestione di progetti e nel coordinamento di team di sviluppo.",
  "esperienze_lavorative": [
    {
      "titolo": "Sviluppatore .NET Senior",
      "azienda": "ABC Solutions",
      "descrizione": "- Responsabile dello sviluppo di un sistema di gestione aziendale basato su ASP.NET Core e C#.\n- Progettazione e implementazione di architetture scalabili e robuste.\n- Coordinamento di un team di sviluppatori, fornendo linee guida tecniche e supporto.\n- Collaborazione con i clienti per comprendere le esigenze e fornire soluzioni personalizzate.\n- Ottimizzazione delle prestazioni e risoluzione di problemi complessi."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea magistrale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Linguaggi di programmazione: C#, HTML, CSS, JavaScript",
    "Framework: .NET, ASP.NET Core, Entity Framework, Xamarin",
    "Database: Microsoft SQL Server, Oracle, MongoDB",
    "Strumenti: Visual Studio, Git, Azure DevOps, Docker",
    "Conoscenza approfondita dei principi di progettazione software e dei design pattern",
    "Esperienza nell'integrazione di sistemi di terze parti e API",
    "Competenza nella gestione di progetti e nello sviluppo Agile (Scrum, Kanban)"
  ],
  "certificazioni": [
    "Certificazione Microsoft Certified: Azure Developer Associate",
    "Certificazione Microsoft Certified: .NET Developer"
  ],
  "pubblicazioni_contributi": [
    "Autore di articoli tecnici su blog e riviste specializzate",
    "Relatore in conferenze e meetup sulla programmazione .NET"
  ]
}
"""
cv2 = """{
  "nome": "Nome Cognome",
  "sommario": "Professionista infrastrutturale altamente qualificato con più di 5 anni di esperienza nella gestione e nell'ottimizzazione delle infrastrutture IT. Esperienza nella progettazione e implementazione di soluzioni infrastrutturali complesse.",
  "esperienze_lavorative": [
    {
      "titolo": "Responsabile infrastrutturale",
      "azienda": "ABC Solutions",
      "descrizione": "- Progettazione e implementazione di architetture di rete e infrastrutture cloud.\n- Gestione di team infrastrutturali, fornendo orientamenti tecnici e supporto.\n- Sicurezza informatica: implementazione di politiche di sicurezza, gestione degli accessi, monitoraggio delle minacce.\n- Collaborazione con i reparti di sviluppo e di supporto per la risoluzione dei problemi."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea magistrale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Sistemi operativi: Windows Server, Linux (Red Hat, Ubuntu)",
    "Networking: TCP/IP, VLAN, VPN, firewall, load balancing",
    "Cloud computing: Amazon Web Services (AWS), Microsoft Azure",
    "Sicurezza informatica: monitoraggio delle minacce, access control, penetration testing",
    "Strumenti: Active Directory, PowerShell, Ansible, Terraform",
    "Conoscenza approfondita della gestione dei progetti infrastrutturali e delle best practice di ITIL"
  ],
  "certificazioni": [
    "Certificazione Cisco Certified Network Associate (CCNA)",
    "Certificazione Microsoft Certified: Azure Administrator Associate",
    "Certificazione ITIL Foundation"
  ]
}
"""
cvAnalistaMag5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Analista funzionale esperto con oltre 5 anni di esperienza nella raccolta, analisi e gestione dei requisiti per progetti software complessi. Capacità dimostrate nel tradurre i requisiti aziendali in soluzioni funzionali e nel coordinare le attività di implementazione.",
  "esperienze_lavorative": [
    {
      "titolo": "Senior Business Analyst presso ABC Solutions",
      "descrizione": "- Collaborazione con gli stakeholder chiave per definire i requisiti funzionali di progetti complessi.\n- Analisi dei processi aziendali, identificazione delle lacune e proposta di soluzioni ottimali.\n- Coordinamento con il team di sviluppo per garantire il completamento dei deliverable secondo i tempi previsti.\n- Gestione del ciclo di vita dei requisiti, dalla raccolta all'implementazione e al testing."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea magistrale in Ingegneria Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Analisi dei requisiti: raccolta, documentazione e gestione dei requisiti",
    "Modellazione dei processi aziendali: diagrammi di flusso, UML",
    "Metodologie di sviluppo software: Waterfall, Agile",
    "Strumenti: Microsoft Office Suite, JIRA, Confluence",
    "Gestione dei cambiamenti e gestione degli stakeholder",
    "Conoscenza approfondita della progettazione e sviluppo software",
    "Competenze di problem-solving e pensiero critico"
  ],
  "certificazioni": [
    "Certificazione IIBA Certified Business Analysis Professional (CBAP)",
    "Certificazione Agile Certified Practitioner (PMI-ACP)"
  ]
}
"""

cvSvilMin5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Sviluppatore .NET altamente motivato con solide competenze di programmazione e problem-solving. Orientato all'apprendimento continuo e alla collaborazione per raggiungere risultati di successo.",
  "esperienze_lavorative": [
    {
      "titolo": "Stage presso ABC Solutions",
      "descrizione": "- Sviluppo di applicazioni basate su .NET Framework utilizzando C#.\n- Collaborazione con il team di sviluppo per la realizzazione di funzionalità software.\n- Test e risoluzione dei problemi."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea triennale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Linguaggi di programmazione: C#, HTML, CSS, JavaScript",
    "Framework: .NET Framework, ASP.NET, Entity Framework",
    "Database: Microsoft SQL Server, MySQL",
    "Strumenti: Visual Studio, Git",
    "Conoscenza dei concetti di base di progettazione e sviluppo software"
  ]
}
"""

cvInfraMin5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Professionista infrastrutturale con passione per la tecnologia e solide competenze nella gestione delle infrastrutture IT. Orientato alla risoluzione dei problemi e al miglioramento continuo.",
  "esperienze_lavorative": [
    {
      "titolo": "Tecnico infrastrutturale presso XYZ Corporation",
      "descrizione": "- Configurazione e manutenzione di server e reti aziendali.\n- Supporto tecnico agli utenti e risoluzione dei problemi infrastrutturali.\n- Monitoraggio delle prestazioni e ottimizzazione delle risorse di rete."
    }
  ],
  "istruzione": [
    {
      "titolo": "Diploma tecnico in Informatica",
      "istituto": "Scuola ABC"
    }
  ],
  "competenze_tecniche": [
    "Sistemi operativi: Windows Server, Linux",
    "Networking: TCP/IP, DNS, DHCP",
    "Sicurezza informatica: firewall, antivirus, backup",
    "Strumenti: Active Directory, VMware, Wireshark",
    "Competenze di base nella gestione dei progetti"
  ]
}
"""

cvAnalistaMin5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Analista funzionale motivato con solide competenze analitiche e di problem-solving. Capacità di comprendere le esigenze aziendali e tradurle in specifiche funzionali.",
  "esperienze_lavorative": [
    {
      "titolo": "Junior Analyst presso ABC Solutions",
      "descrizione": "- Raccolta e analisi dei requisiti funzionali in progetti software.\n- Supporto nella progettazione di soluzioni e nella stesura di documentazione.\n- Collaborazione con il team di sviluppo per garantire l'allineamento alle specifiche."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea triennale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Analisi dei requisiti: raccolta, documentazione e gestione dei requisiti",
    "Modellazione dei processi aziendali: diagrammi di flusso, UML",
    "Metodologie di sviluppo software: Waterfall, Agile",
    "Strumenti: Microsoft Office Suite, JIRA",
    "Conoscenza dei principi di base dell'analisi funzionale"
  ]
}
"""


cvSvil5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Sviluppatore .NET con 5 anni di esperienza nella progettazione e implementazione di soluzioni software. Esperto nella scrittura di codice pulito e mantenibile, con una solida comprensione dei principi di sviluppo software.",
  "esperienze_lavorative": [
    {
      "titolo": "Sviluppatore .NET presso ABC Solutions",
      "descrizione": "- Progettazione e sviluppo di applicazioni aziendali basate su .NET utilizzando C#.\n- Collaborazione con il team di sviluppo per la realizzazione di funzionalità complesse.\n- Migrazione di applicazioni legacy a nuove architetture tecnologiche.\n- Implementazione di test automatici per garantire la qualità del software."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea magistrale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Linguaggi di programmazione: C#, HTML, CSS, JavaScript",
    "Framework: .NET, ASP.NET MVC, Entity Framework",
    "Database: Microsoft SQL Server, MySQL",
    "Strumenti: Visual Studio, Git, Azure DevOps",
    "Conoscenza approfondita dei principi di progettazione software e dei design pattern",
    "Esperienza nello sviluppo di applicazioni Web e servizi RESTful",
    "Competenza nella gestione di progetti e nello sviluppo Agile (Scrum)"
  ],
  "certificazioni": [
    "Certificazione Microsoft Certified: Azure Developer Associate",
    "Certificazione Microsoft Certified: .NET Developer"
  ]
}
"""
cvInfra5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Professionista infrastrutturale con 5 anni di esperienza nella gestione e nell'ottimizzazione delle infrastrutture IT. Competenze avanzate nella progettazione e implementazione di soluzioni infrastrutturali complesse.",
  "esperienze_lavorative": [
    {
      "titolo": "Responsabile infrastrutturale presso ABC Solutions",
      "descrizione": "- Progettazione e implementazione di architetture di rete e infrastrutture cloud.\n- Gestione di team infrastrutturali e coordinamento delle attività.\n- Monitoraggio delle prestazioni e risoluzione dei problemi infrastrutturali.\n- Implementazione di soluzioni per la sicurezza e il disaster recovery."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea triennale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Sistemi operativi: Windows Server, Linux (Red Hat, Ubuntu)",
    "Networking: TCP/IP, VLAN, VPN, firewall, load balancing",
    "Cloud computing: Amazon Web Services (AWS), Microsoft Azure",
    "Sicurezza informatica: monitoraggio delle minacce, access control, penetration testing",
    "Strumenti: Active Directory, PowerShell, Ansible, Terraform",
    "Conoscenza approfondita della gestione dei progetti infrastrutturali e delle best practice di ITIL"
  ],
  "certificazioni": [
    "Certificazione Cisco Certified Network Professional (CCNP)",
    "Certificazione Microsoft Certified: Azure Solutions Architect Expert",
    "Certificazione ITIL Intermediate"
  ]
}
"""
cvAnalista5Anni = """{
  "nome": "Nome Cognome",
  "indirizzo": "Indirizzo",
  "telefono": "Numero di telefono",
  "email": "Indirizzo email",
  "sommario": "Analista funzionale con 5 anni di esperienza nella raccolta e analisi dei requisiti per progetti software complessi. Abilità nell'interfacciarsi con gli stakeholder e nella traduzione dei requisiti in specifiche funzionali.",
  "esperienze_lavorative": [
    {
      "titolo": "Analista funzionale presso ABC Solutions",
      "descrizione": "- Collaborazione con team multidisciplinari per definire i requisiti funzionali di progetti software.\n- Analisi dei processi aziendali e identificazione delle aree di miglioramento.\n- Stesura di documenti di analisi funzionale e specifiche dei requisiti.\n- Supporto al team di sviluppo durante la fase di implementazione."
    }
  ],
  "istruzione": [
    {
      "titolo": "Laurea magistrale in Informatica",
      "universita": "Università XYZ"
    }
  ],
  "competenze_tecniche": [
    "Analisi dei requisiti: raccolta, documentazione e gestione dei requisiti",
    "Modellazione dei processi aziendali: diagrammi di flusso, UML",
    "Metodologie di sviluppo software: Waterfall, Agile",
    "Strumenti: Microsoft Office Suite, JIRA, Confluence",
    "Esperienza nella gestione dei cambiamenti e nella gestione degli stakeholder",
    "Conoscenza dei principi di base della progettazione e dello sviluppo software"
  ],
  "certificazioni": [
    "Certificazione IIBA Certified Business Analysis Professional (CBAP)",
    "Certificazione Agile Certified Practitioner (PMI-ACP)"
  ]
}
"""

# Create DataFrame with all the cv dictionary
cvRepo = { 'CV_Description' : ["CV Sviluppatore > 5anni", "CV Infrastrutturale > 5anni", "CV Analista > 5anni", "CV Sviluppatore < 5", "CV Infrastrutturale < 5", "CV Analista < 5",
                               "CV Sviluppatore 5anni", "CV Infrastrutturale 5anni", "CV Analista 5anni"],
           'CV' : [cv1, cv2, cvAnalistaMag5Anni, cvSvil5Anni, cvInfra5Anni, cvAnalista5Anni, cvSvilMin5Anni, cvInfraMin5Anni, cvAnalistaMin5Anni]}
dfCvRepo = pd.DataFrame(cvRepo)

# add embedding column with embedding
dfCvRepo['embedding'] = dfCvRepo['CV'].apply(lambda x: embeddings.embed_query(x))
dfCvRepo['embedding_len'] = dfCvRepo['embedding'].apply(lambda x: len(x))


# Let's make a search on cv df based on a particular need
request = "Sviluppatore .net con almeno cinque anni di esperienza"

# get embedding for request
request_embedding = embeddings.embed_query(request)

# get cosine similarity between request and all cv descriptions in DataFrame
similarities = dfCvRepo['embedding'].apply(lambda x: cosine_similarity(x, request_embedding))
dfCvRepo.head()

# get cosine similarity between request and all libraries descriptions
get_similarity_from_text(cv1, request, "CV Sviluppatore > 5anni", "Richiesta", False)
get_similarity_from_text(cv2, request, "CV Infrastrutturale > 5anni", "Richiesta", False)
get_similarity_from_text(cvAnalistaMag5Anni, request, "CV Analista > 5anni", "Richiesta", False)
get_similarity_from_text(cvSvilMin5Anni, request, "CV Sviluppatore < 5", "Richiesta", False)
get_similarity_from_text(cvInfraMin5Anni, request, "CV Infrastrutturale < 5", "Richiesta", False)
get_similarity_from_text(cvAnalistaMin5Anni, request, "CV Analista < 5", "Richiesta", False)
get_similarity_from_text(cvSvil5Anni, request, "CV Sviluppatore 5anni", "Richiesta", False)
get_similarity_from_text(cvInfra5Anni, request, "CV Infrastrutturale 5anni", "Richiesta", False)
get_similarity_from_text(cvAnalista5Anni, request, "CV Analista 5anni", "Richiesta", False)


# combine library_name from df and similiaries and sort ascending by similarity
recommendations = pd.concat([dfCvRepo['CV_Description'], similarities, dfCvRepo['embedding_len']], axis=1).sort_values(by='embedding', ascending=False)
recommendations.head(10)


Somiglianza tra 'CV Sviluppatore > 5anni' e 'Richiesta': 0.8304086743981158
Somiglianza tra 'CV Infrastrutturale > 5anni' e 'Richiesta': 0.7702719343175558
Somiglianza tra 'CV Analista > 5anni' e 'Richiesta': 0.7652037247436577
Somiglianza tra 'CV Sviluppatore < 5' e 'Richiesta': 0.8178658100207891
Somiglianza tra 'CV Infrastrutturale < 5' e 'Richiesta': 0.7563457259710038
Somiglianza tra 'CV Analista < 5' e 'Richiesta': 0.7602890681533705
Somiglianza tra 'CV Sviluppatore 5anni' e 'Richiesta': 0.8184438570988692
Somiglianza tra 'CV Infrastrutturale 5anni' e 'Richiesta': 0.7580876588298581
Somiglianza tra 'CV Analista 5anni' e 'Richiesta': 0.7680433338056389


,CV_Description,embedding,embedding_len
0,CV Sviluppatore > 5anni,0.830409,1536
3,CV Sviluppatore < 5,0.818444,1536
6,CV Sviluppatore 5anni,0.817866,1536
1,CV Infrastrutturale > 5anni,0.770272,1536
5,CV Analista < 5,0.768043,1536
2,CV Analista > 5anni,0.765204,1536
8,CV Analista 5anni,0.760289,1536
4,CV Infrastrutturale < 5,0.758088,1536
7,CV Infrastrutturale 5anni,0.756346,1536


In [6]:

print(similarities)

0    0.830409
1    0.770272
2    0.765204
3    0.818444
4    0.758088
5    0.768043
6    0.817866
7    0.756346
8    0.760289
Name: embedding, dtype: float64
